<a href="https://colab.research.google.com/github/avc9/NLPlay-with-Transformers--SoC-2021/blob/main/GPT2_and_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel,T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

device="cuda:0" if torch.cuda.is_available() else "cpu"

print(device)

cuda:0


In [3]:
tokenizer = T5Tokenizer.from_pretrained('t5-large')
model = T5ForConditionalGeneration.from_pretrained('t5-large')
model=model.to(device)



In [13]:
input_ids = tokenizer.encode("summarize: Hello, my dog is cute", return_tensors="pt")  # Batch size 1
outputs = model.generate(input_ids.to(device),max_length=200,top_k=50,temperature=0.8,no_repeat_ngram_size=2)
print(outputs.size())

tokenizer.decode(outputs[0],skip_special_tokens=True)

torch.Size([1, 52])


'my dog is cute, my cat is funny and my dachshund is a poodle. i love dogs and he is so sweet - if u like dogs, ill take him home!'

In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)
model=model.to(device)

In [28]:
science_file_path="/content/drive/MyDrive/entertainment articles.txt"
file1 = open(science_file_path, 'r')
count = 0
list1=[]
# Using for loop
print("Using for loop")
for line in file1:
    count += 1
    print(f'{count}')
    list1.append(line.strip())
 
# Closing files
file1.close()

Using for loop
1
2
3
4
5
6
7
8
9
10


In [8]:
def text_generator(original_text,char_limit=2000):
  intial_text=original_text.split()[:50]
  print(intial_text)
  text=''
  for i in intial_text:
    text+=i+" "

  print(len(text),text)

  ip=tokenizer.encode(text,return_tensors='pt')
  #ip1.shape
  ip=ip.to(device)
  ans=model.generate(ip,max_length=1000,do_sample=True,top_k=100,no_repeat_ngram_size=2)
  #text.join(' ')
  text=tokenizer.decode(ans[0],skip_special_tokens=True)


  while (len(text.split())<char_limit):
    print(len(text.split()))
    new_ip_text=" "
    new_ip_text=new_ip_text.join(text.split()[-500:])
    ip=tokenizer.encode(new_ip_text,return_tensors='pt')
    #ip1.shape
    ip=ip.to(device)
    ans=model.generate(ip,max_length=1000,do_sample=True,top_k=100,no_repeat_ngram_size=2)
    #text.join(' ')
    text+=tokenizer.decode(ans[0],skip_special_tokens=True)

  return text




In [49]:
def text_generatorT5(original_text,char_limit=2000):
  intial_text=original_text.split()[:50]
  print(intial_text)
  text=''
  for i in intial_text:
    text+=i+" "


  text="summarize: "+text
  print(len(text),text)

  ip=tokenizer.encode(text,return_tensors='pt')
  #ip1.shape
  ip=ip.to(device)
  ans=model.generate(ip,max_length=1000,do_sample=True,top_k=100,no_repeat_ngram_size=2)
  #text.join(' ')
  text=tokenizer.decode(ans[0],skip_special_tokens=True)


  while (len(text.split())<char_limit):
    #print(len(text.split()))
    mid=int(len(text.split())/2)
    new_ip_text=" "
    new_ip_text=new_ip_text.join(text.split()[-mid:])
    print(new_ip_text)
    new_ip_text="summarize: "+new_ip_text
    ip=tokenizer.encode(new_ip_text,return_tensors='pt')
    #ip1.shape
    ip=ip.to(device)
    ans=model.generate(ip,max_length=1000,do_sample=True,top_k=100,no_repeat_ngram_size=2)
    #text.join(' ')
    add_text=" "
    add_text=add_text.join(tokenizer.decode(ans[0],skip_special_tokens=True).split()[mid:])


    text+=tokenizer.decode(ans[0],skip_special_tokens=True)

  return text

In [50]:
gen_text=text_generatorT5(list1[0])
print(gen_text)

['Entertainment', 'is', 'a', 'form', 'of', 'activity', 'that', 'holds', 'the', 'attention', 'and', 'interest', 'of', 'an', 'audience', 'or', 'gives', 'pleasure', 'and', 'delight.', 'It', 'can', 'be', 'an', 'idea', 'or', 'a', 'task,', 'but', 'is', 'more', 'likely', 'to', 'be', 'one', 'of', 'the', 'activities', 'or', 'events', 'that', 'have', 'developed', 'over', 'thousands', 'of', 'years', 'specifically', 'for', 'the']
282 summarize: Entertainment is a form of activity that holds the attention and interest of an audience or gives pleasure and delight. It can be an idea or a task, but is more likely to be one of the activities or events that have developed over thousands of years specifically for the 
it is too obvious to see 'the message' entertainment can make santa amara feel beautiful and alive again, and is ideal for gandhi's journey: she is very proud to represent her homeland thailand in these pages / it' retells the
very proud to represent her homeland thailand in these pages / i

In [51]:
bleu(list1[0],gen_text)

0.12175114371305808

In [ ]:
#len(list1[2].split())
#len(gen_text.split())


In [35]:
def bleu(ref, gen,limit=1800):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen.split()[:limit]:
        gen_bleu.append(l)
    for l in ref.split()[:limit]:
        ref_bleu.append(l)

    #print(ref_bleu[50:100],"\n",gen_bleu[50:100])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
#bleu(list1[2],gen_text)

In [ ]:
'''!pip install newspaper3k
import newspaper
from newspaper import Article
import nltk

nltk.download('punkt')
url= 'https://www.financialexpress.com/money/7th-pay-commission-dearness-allowance-hike-latest-news-today-modi-govt-da-dr-increase-for-central-government-employees-pensioners-details/2290059/'

article = Article(url, language="en")
article.download() 
article.parse()''' 
#article.nlp()

'!pip install newspaper3k\nimport newspaper\nfrom newspaper import Article\nimport nltk\n\nnltk.download(\'punkt\')\nurl= \'https://www.financialexpress.com/money/7th-pay-commission-dearness-allowance-hike-latest-news-today-modi-govt-da-dr-increase-for-central-government-employees-pensioners-details/2290059/\'\n\narticle = Article(url, language="en")\narticle.download() \narticle.parse()'

In [ ]:
#print(len(article.text.split()))

In [ ]:
bleu_scores=[]
for para in list1:
  generated=text_generator(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/finance articles generated.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores.append(bleu(para,generated))
  


['A', 'cryptocurrency,', 'crypto-currency,', 'or', 'crypto', 'is', 'a', 'digital', 'asset', 'designed', 'to', 'work', 'as', 'a', 'medium', 'of', 'exchange', 'wherein', 'individual', 'coin', 'ownership', 'records', 'are', 'stored', 'in', 'a', 'ledger', 'existing', 'in', 'a', 'form', 'of', 'a', 'computerized', 'database', 'using', 'strong', 'cryptography', 'to', 'secure', 'transaction', 'records,', 'to', 'control', 'the', 'creation', 'of', 'additional', 'coins,', 'and']
322 A cryptocurrency, crypto-currency, or crypto is a digital asset designed to work as a medium of exchange wherein individual coin ownership records are stored in a ledger existing in a form of a computerized database using strong cryptography to secure transaction records, to control the creation of additional coins, and 
805
1603
['Finance', 'is', 'a', 'term', 'for', 'matters', 'regarding', 'the', 'management,', 'creation,', 'and', 'study', 'of', 'money', 'and', 'investments.Specifically,', 'it', 'deals', 'with', 'the

In [ ]:
bleu_scores_enter=[]
for para in list1:
  generated=text_generator(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/entertainment articles generated.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores_enter.append(bleu(para,generated))

331
765
1273
1786
['or', 'part', 'of', 'the', 'audience,', 'or', 'they', 'may', 'swap', 'roles', 'even', 'during', 'the', 'course', 'of', 'one', 'entertainment.', 'In', 'the', 'court', 'at', 'the', 'Palace', 'of', 'Versailles,', '"thousands', 'of', 'courtiers,', 'including', 'men', 'and', 'women', 'who', 'inhabited', 'its', 'apartments,', 'acted', 'as', 'both', 'performers', 'and', 'spectators', 'in', 'daily', 'rituals', 'that', 'reinforced', 'the', 'status', 'hierarchy".Like']
314 or part of the audience, or they may swap roles even during the course of one entertainment. In the court at the Palace of Versailles, "thousands of courtiers, including men and women who inhabited its apartments, acted as both performers and spectators in daily rituals that reinforced the status hierarchy".Like 
851
1729
['primary', 'purpose.The', 'meaning', 'of', 'the', 'word', '"comedy"', 'and', 'the', "audience's", 'expectations', 'of', 'it', 'have', 'changed', 'over', 'time', 'and', 'vary', 'according',

In [ ]:
import re
bleu_scores_sci=[]
for para in list1:
  generated=text_generator(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/science articles generated.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores_sci.append(bleu(para,generated))

['The', 'Big', 'Bang', 'is', 'actually', 'not', 'a', '"theory"', 'at', 'all,', 'but', 'rather', 'a', 'scenario', 'or', 'model', 'about', 'the', 'early', 'moments', 'of', 'our', 'universe,', 'for', 'which', 'the', 'evidence', 'is', 'overwhelming.', 'It', 'is', 'a', 'common', 'misconception', 'that', 'the', 'Big', 'Bang', 'was', 'the', 'origin', 'of', 'the', 'universe.', 'In', 'reality,', 'the', 'Big', 'Bang', 'scenario']
273 The Big Bang is actually not a "theory" at all, but rather a scenario or model about the early moments of our universe, for which the evidence is overwhelming. It is a common misconception that the Big Bang was the origin of the universe. In reality, the Big Bang scenario 
393
1227
['The', 'Cosmic', 'Background', 'Explorer', '(COBE)', 'satellite', 'was', 'launched', 'in', '1989,', 'twenty', 'five', 'years', 'after', 'the', 'discovery', 'of', 'the', 'microwave', 'background', 'radiation', 'in', '1964.', 'In', '1992,', 'the', 'COBE', 'team', 'announced', 'that', 'they

In [ ]:
print("Finance bleu scores:")
for i in bleu_scores:
  print(i)

Finance bleu scores:
0.29372960133044873
0.2170733578395206
0.29372960133044873
0.29372960133044873
0.29372960133044873
0.12175114371305808
0.28015193218702505
0.1801413562081085
0.12175114371305808
0.24350601878055653


In [ ]:
print("Entertainment bleu scores:")
for i in bleu_scores_enter:
  print(i)

Entertainment bleu scores:
0.1801413562081085
0.3153109568285757
0.24350601878055653
0.24350601878055653
0.1801413562081085
0.24350601878055653
0.12175114371305808
0.2170733578395206
0.2170733578395206
0.1801413562081085


In [ ]:
print("Science bleu scores:")
for i in bleu_scores_sci:
  print(i)

Science bleu scores:
0.1801413562081085
0.26381374036376243
0.1801413562081085
0.12175114371305808
0.28015193218702505
0.12175114371305808
0.24350601878055653
0.28015193218702505
0.26381374036376243
0.12175114371305808


In [21]:
import re
bleu_scores_sci_T5=[]
for para in list1:
  generated=text_generatorT5(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/science articles generated T5.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores_sci_T5.append(bleu(para,generated))

['The', 'Big', 'Bang', 'is', 'actually', 'not', 'a', '"theory"', 'at', 'all,', 'but', 'rather', 'a', 'scenario', 'or', 'model', 'about', 'the', 'early', 'moments', 'of', 'our', 'universe,', 'for', 'which', 'the', 'evidence', 'is', 'overwhelming.', 'It', 'is', 'a', 'common', 'misconception', 'that', 'the', 'Big', 'Bang', 'was', 'the', 'origin', 'of', 'the', 'universe.', 'In', 'reality,', 'the', 'Big', 'Bang', 'scenario']
284 summarize: The Big Bang is actually not a "theory" at all, but rather a scenario or model about the early moments of our universe, for which the evidence is overwhelming. It is a common misconception that the Big Bang was the origin of the universe. In reality, the Big Bang scenario 
['The', 'Cosmic', 'Background', 'Explorer', '(COBE)', 'satellite', 'was', 'launched', 'in', '1989,', 'twenty', 'five', 'years', 'after', 'the', 'discovery', 'of', 'the', 'microwave', 'background', 'radiation', 'in', '1964.', 'In', '1992,', 'the', 'COBE', 'team', 'announced', 'that', 'th

In [26]:
bleu_scores_T5=[]
for para in list1:
  generated=text_generatorT5(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/finance articles generated T5.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores_T5.append(bleu(para,generated))

['A', 'cryptocurrency,', 'crypto-currency,', 'or', 'crypto', 'is', 'a', 'digital', 'asset', 'designed', 'to', 'work', 'as', 'a', 'medium', 'of', 'exchange', 'wherein', 'individual', 'coin', 'ownership', 'records', 'are', 'stored', 'in', 'a', 'ledger', 'existing', 'in', 'a', 'form', 'of', 'a', 'computerized', 'database', 'using', 'strong', 'cryptography', 'to', 'secure', 'transaction', 'records,', 'to', 'control', 'the', 'creation', 'of', 'additional', 'coins,', 'and']
333 summarize: A cryptocurrency, crypto-currency, or crypto is a digital asset designed to work as a medium of exchange wherein individual coin ownership records are stored in a ledger existing in a form of a computerized database using strong cryptography to secure transaction records, to control the creation of additional coins, and 
['Finance', 'is', 'a', 'term', 'for', 'matters', 'regarding', 'the', 'management,', 'creation,', 'and', 'study', 'of', 'money', 'and', 'investments.Specifically,', 'it', 'deals', 'with', 't

In [29]:
bleu_scores_ent_T5=[]
for para in list1:
  generated=text_generatorT5(para)
  generated = re.sub(r'==.*?==+', '', generated)
  generated = generated.replace('\n', '')

  file1=open("/content/drive/MyDrive/entertainment articles generated T5.txt","a")
  file1.write(generated+"\n")
  file1.close()

  bleu_scores_ent_T5.append(bleu(para,generated))

['Entertainment', 'is', 'a', 'form', 'of', 'activity', 'that', 'holds', 'the', 'attention', 'and', 'interest', 'of', 'an', 'audience', 'or', 'gives', 'pleasure', 'and', 'delight.', 'It', 'can', 'be', 'an', 'idea', 'or', 'a', 'task,', 'but', 'is', 'more', 'likely', 'to', 'be', 'one', 'of', 'the', 'activities', 'or', 'events', 'that', 'have', 'developed', 'over', 'thousands', 'of', 'years', 'specifically', 'for', 'the']
282 summarize: Entertainment is a form of activity that holds the attention and interest of an audience or gives pleasure and delight. It can be an idea or a task, but is more likely to be one of the activities or events that have developed over thousands of years specifically for the 
['or', 'part', 'of', 'the', 'audience,', 'or', 'they', 'may', 'swap', 'roles', 'even', 'during', 'the', 'course', 'of', 'one', 'entertainment.', 'In', 'the', 'court', 'at', 'the', 'Palace', 'of', 'Versailles,', '"thousands', 'of', 'courtiers,', 'including', 'men', 'and', 'women', 'who', 'in

In [22]:
print("Science bleu scores T5:")
for i in bleu_scores_sci_T5:
  print(i)

Science bleu scores T5:
0.1801413562081085
0.1801413562081085
0.30528776109824257
0.33905704271921533
0.12175114371305808
0.26381374036376243
0.3241320922202416
0.28015193218702505
0.3153109568285757
0.26381374036376243


In [27]:
print("Finance bleu scores T5:")
for i in bleu_scores_T5:
  print(i)

Finance bleu scores T5:
0.2170733578395206
0.29372960133044873
0.26381374036376243
0.28015193218702505
0.26381374036376243
0.1801413562081085
0.2170733578395206
0.2170733578395206
0.2170733578395206
0.2170733578395206


In [30]:
print("Entertainment bleu scores T5:")
for i in bleu_scores_ent_T5:
  print(i)

Entertainment bleu scores T5:
0.1801413562081085
0.28015193218702505
0.1801413562081085
0.24350601878055653
0.2170733578395206
0.29372960133044873
0.24350601878055653
0.28015193218702505
0.1801413562081085
0.26381374036376243


##Total bleu of Finance

In [ ]:
finance_text=''

file1=open('/content/drive/MyDrive/finance articles.txt','r')

for line in file1:
  finance_text+=line

file1.close()

finance_text_gpt2=''

file1=open('/content/drive/MyDrive/finance articles generated.txt','r')

for line in file1:
  finance_text_gpt2+=line

file1.close()

finance_text_T5=''

file1=open('/content/drive/MyDrive/finance articles generated T5.txt','r')

for line in file1:
  finance_text_T5+=line

file1.close()



In [34]:
print(len(finance_text.split()),len(finance_text_gpt2.split()),len(finance_text_T5.split()))

21000 25913 20207


In [37]:
bleu_finance_gpt2=bleu(finance_text,finance_text_gpt2,20000)
bleu_finance_T5=bleu(finance_text,finance_text_T5,20000)

print(bleu_finance_gpt2)
print(bleu_finance_T5)

0.26381374036376243
0.2170733578395206


##Total bleu of Entertainment

In [39]:
entertainment_text=''

file1=open('/content/drive/MyDrive/entertainment articles.txt','r')

for line in file1:
  entertainment_text+=line

file1.close()

entertainment_text_gpt2=''

file1=open('/content/drive/MyDrive/entertainment articles generated.txt','r')

for line in file1:
  entertainment_text_gpt2+=line

file1.close()

entertainment_text_T5=''

file1=open('/content/drive/MyDrive/entertainment articles generated T5.txt','r')

for line in file1:
  entertainment_text_T5+=line

file1.close()

In [40]:
print(len(entertainment_text.split()),len(entertainment_text_gpt2.split()),len(entertainment_text_T5.split()))

21000 24129 20310


In [41]:
bleu_entertainment_gpt2=bleu(entertainment_text,entertainment_text_gpt2,20000)
bleu_entertainment_T5=bleu(entertainment_text,entertainment_text_T5,20000)

print(bleu_entertainment_gpt2)
print(bleu_entertainment_T5)

0.24350601878055653
0.24350601878055653


## Total bleu of Science

In [42]:
science_text=''

file1=open('/content/drive/MyDrive/science articles.txt','r')

for line in file1:
  science_text+=line

file1.close()

science_text_gpt2=''

file1=open('/content/drive/MyDrive/science articles generated.txt','r')

for line in file1:
  science_text_gpt2+=line

file1.close()

science_text_T5=''

file1=open('/content/drive/MyDrive/science articles generated T5.txt','r')

for line in file1:
  science_text_T5+=line

file1.close()

In [43]:
print(len(science_text.split()),len(science_text_gpt2.split()),len(science_text_T5.split()))

20793 46924 30499


In [45]:
bleu_science_gpt2=bleu(science_text,science_text_gpt2,20000)
bleu_science_T5=bleu(science_text,science_text_T5,20000)

print(bleu_science_gpt2)
print(bleu_science_T5)

0.1801413562081085
0.12175114371305808


In [ ]:
!pip install wikipedia

In [ ]:
# Import package
import wikipedia
# Specify the title of the Wikipedia page
wiki = wikipedia.page('Thor (Marvel Comics)')
# Extract the plain text content of the page
wiki_text = wiki.content
#print(wiki_text)

# Import package
import re
# Clean text
wiki_text = re.sub(r'==.*?==+', '', wiki_text)
wiki_text = wiki_text.replace('\n', '')
print(wiki_text[:2000])
print(len(wiki_text.split()))
wiki_text=" ".join(wiki_text.split()[:2100].copy())
print(wiki_text)

Thor Odinson, or simply Thor is a fictional character appearing in American comic books published by Marvel Comics. The character, which is based on the Norse deity of the same name, is the Asgardian god of thunder who possesses the enchanted hammer, Mjolnir, which grants him the ability to fly and manipulate weather amongst his other superhuman attributes.Debuting in the Silver Age of Comic Books, the character first appeared in Journey into Mystery #83 (August 1962) and was created by penciller-plotter Jack Kirby, editor-plotter Stan Lee, and scripter Larry Lieber. The character has starred in several ongoing series and limited series, and is a founding member of the superhero team the Avengers, appearing in all volumes of the series. The character has been seen in Marvel Comics merchandise, animated television series, movies, video games, clothing and toys. Thor has a host of supporting characters and enemies.Chris Hemsworth portrays the character in several Marvel Cinematic Univers

In [ ]:
wikipedia.search('Thor')

In [ ]:
'''finance_file_path="/content/drive/MyDrive/entertainment articles.txt"
file1=open(finance_file_path,"a")
file1.write(wiki_text+"\n")
file1.close()'''